In [1]:
# %pip install snowflake-connector-python
# %pip install pyarrow==8.0.0
# %pip install pandas
# %pip install confluent-kafka
# %pip install python-dotenv

In [2]:
import snowflake.connector
from snowflake_helpers import *
import re
import pandas as pd
from transformation_helpers import *

In [3]:
conn = connect_to_snowflake()
cs = conn.cursor()

Connected to Snowflake with user: admin


In [4]:
create_production_schema(cs)
use_production_schema(cs)

Schema: yusra_stories_production created.


In [5]:
df = cs.execute('SELECT * FROM yusra_stories_staging.logs').fetch_pandas_all()
df.head(5)

,log_id,log
0,1,ride 1 --------- beginning of a new ride\n
1,2,ride 1 2022-10-06 15:49:21.453226 mendoza v9: ...
2,3,ride 1 2022-10-06 15:49:21.953902 mendoza v9: ...
3,4,ride 1 2022-10-06 15:49:22.454586 mendoza v9: ...
4,5,ride 1 2022-10-06 15:49:22.955260 mendoza v9: ...


In [6]:
formatted_df = get_joined_formatted_df(df)

at the start df is <class 'pandas.core.frame.DataFrame'>


/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:62: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if birthday > today:
/Users/harrymay/Desktop/coursework/deloton-group-project/transformation_helpers.py:62: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if birthday > today:


In [7]:
formatted_df.head(3)

,log_id,log,ride_id,is_new_ride,is_info,is_system,time,user_id,name,gender,...,address,email_address,account_created,bike_serial,original_source,duration_secs,heart_rate,resistance,rpm,power
0,1,ride 1 --------- beginning of a new ride\n,1,True,False,False,NaT,NaN,None,None,...,None,None,NaT,None,None,NaN,NaN,NaN,NaN,NaN
1,2,ride 1 2022-10-06 15:49:21.453226 mendoza v9: ...,1,False,False,False,2022-10-06 15:49:21.453226,NaN,None,None,...,None,None,NaT,None,None,NaN,NaN,NaN,NaN,NaN
2,3,ride 1 2022-10-06 15:49:21.953902 mendoza v9: ...,1,False,False,True,2022-10-06 15:49:21.953902,4612.0,Mrs Bethany Welch,female,...,"Flat 3,Jeremy rapid,Evanshaven,S2 9DH",mrs_b@gmail.com,2022-02-27,SN0000,direct,NaN,NaN,NaN,NaN,NaN


In [8]:
user_df = get_users_df(formatted_df)
user_df.head()

,name,gender,date_of_birth,age,height_cm,weight_kg,address,email_address,account_created,bike_serial,original_source
user_id,,,,,,,,,,,
4612.0,Mrs Bethany Welch,female,1968-09-20,54.0,188.0,60.0,"Flat 3,Jeremy rapid,Evanshaven,S2 9DH",mrs_b@gmail.com,2022-02-27,SN0000,direct
4614.0,Jill Gill,female,1996-11-11,25.0,180.0,61.0,"Studio 4,Carly valleys,West George,RH72 6JL",j_g86@gmail.com,2022-03-02,SN0000,google ads
4616.0,Mr Leon Taylor,male,1966-09-29,56.0,171.0,58.0,"Studio 82,Lloyd terrace,Allenport,PL6E 1YN",mr_leon@hotmail.com,2021-12-19,SN0000,direct
4617.0,Mitchell Tomlinson,male,1967-05-23,55.0,151.0,36.0,"520 Amy expressway,East Richardland,BH0V 6HL",mitchell_t@gmail.com,2022-03-15,SN0000,google ads


## RIDE DF

In [9]:
formatted_df.head(1)

,log_id,log,ride_id,is_new_ride,is_info,is_system,time,user_id,name,gender,...,address,email_address,account_created,bike_serial,original_source,duration_secs,heart_rate,resistance,rpm,power
0,1,ride 1 --------- beginning of a new ride\n,1,True,False,False,NaT,NaN,None,None,...,None,None,NaT,None,None,NaN,NaN,NaN,NaN,NaN


In [32]:
staging_ride_columns =  ['ride_id', 'user_id', 'time', 'duration_secs', 'heart_rate', 'resistance', 'power', 'rpm']

In [11]:
staging_rides = formatted_df[staging_ride_columns]
staging_rides

,ride_id,user_id,time,duration_secs,heart_rate,resistance,power,rpm
0,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,1,NaN,2022-10-06 15:49:21.453226,NaN,NaN,NaN,NaN,NaN
2,1,4612.0,2022-10-06 15:49:21.953902,NaN,NaN,NaN,NaN,NaN
3,1,NaN,2022-10-06 15:49:22.454586,1.0,NaN,0.0,NaN,NaN
4,1,NaN,2022-10-06 15:49:22.955260,NaN,0.0,NaN,3.044161,27.0
...,...,...,...,...,...,...,...,...
17632,17,NaN,2022-10-06 18:16:37.116680,NaN,87.0,NaN,8.207265,36.0
17633,17,NaN,2022-10-06 18:16:37.617353,519.0,NaN,30.0,NaN,NaN
17634,17,NaN,2022-10-06 18:16:38.118052,NaN,87.0,NaN,7.837602,35.0
17635,17,NaN,2022-10-06 18:16:38.618768,520.0,NaN,30.0,NaN,NaN


total duration

In [13]:
from datetime import timedelta
staging_rides['total_duration'] = staging_rides.groupby(by=['ride_id'], dropna=False)['duration_secs'].transform('max')
staging_rides['total_duration'] = staging_rides['total_duration'].apply(lambda x: str(timedelta(seconds=x)))
staging_rides

/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1745106254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['total_duration'] = staging_rides.groupby(by=['ride_id'], dropna=False)['duration_secs'].transform('max')
/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1745106254.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['total_duration'] = staging_rides['total_duration'].apply(lambda x: str(timedelta(seconds=x)))


,ride_id,user_id,time,duration_secs,heart_rate,resistance,power,rpm,total_duration
0,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0:08:37
1,1,NaN,2022-10-06 15:49:21.453226,NaN,NaN,NaN,NaN,NaN,0:08:37
2,1,4612.0,2022-10-06 15:49:21.953902,NaN,NaN,NaN,NaN,NaN,0:08:37
3,1,NaN,2022-10-06 15:49:22.454586,1.0,NaN,0.0,NaN,NaN,0:08:37
4,1,NaN,2022-10-06 15:49:22.955260,NaN,0.0,NaN,3.044161,27.0,0:08:37
...,...,...,...,...,...,...,...,...,...
17632,17,NaN,2022-10-06 18:16:37.116680,NaN,87.0,NaN,8.207265,36.0,0:08:40
17633,17,NaN,2022-10-06 18:16:37.617353,519.0,NaN,30.0,NaN,NaN,0:08:40
17634,17,NaN,2022-10-06 18:16:38.118052,NaN,87.0,NaN,7.837602,35.0,0:08:40
17635,17,NaN,2022-10-06 18:16:38.618768,520.0,NaN,30.0,NaN,NaN,0:08:40


Total power

In [14]:
staging_rides['total_power_kilojoules'] = staging_rides.groupby('ride_id', dropna=False)['power'].transform('sum')
staging_rides['total_power_kilojoules'] = staging_rides['total_power_kilojoules'].apply(lambda x: round(x/1000, 2))
staging_rides

/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/4034747580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['total_power_kilojoules'] = staging_rides.groupby('ride_id', dropna=False)['power'].transform('sum')
/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/4034747580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['total_power_kilojoules'] = staging_rides['total_power_kilojoules'].apply(lambda x: round(x/1000, 2))


,ride_id,user_id,time,duration_secs,heart_rate,resistance,power,rpm,total_duration,total_power_kilojoules
0,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51
1,1,NaN,2022-10-06 15:49:21.453226,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51
2,1,4612.0,2022-10-06 15:49:21.953902,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51
3,1,NaN,2022-10-06 15:49:22.454586,1.0,NaN,0.0,NaN,NaN,0:08:37,13.51
4,1,NaN,2022-10-06 15:49:22.955260,NaN,0.0,NaN,3.044161,27.0,0:08:37,13.51
...,...,...,...,...,...,...,...,...,...,...
17632,17,NaN,2022-10-06 18:16:37.116680,NaN,87.0,NaN,8.207265,36.0,0:08:40,6.66
17633,17,NaN,2022-10-06 18:16:37.617353,519.0,NaN,30.0,NaN,NaN,0:08:40,6.66
17634,17,NaN,2022-10-06 18:16:38.118052,NaN,87.0,NaN,7.837602,35.0,0:08:40,6.66
17635,17,NaN,2022-10-06 18:16:38.618768,520.0,NaN,30.0,NaN,NaN,0:08:40,6.66


max heart rate

In [15]:
staging_rides['max_heart_rate_bpm'] = staging_rides.groupby('ride_id', dropna=False)['heart_rate'].transform('max')
staging_rides['max_heart_rate_bpm'] = staging_rides['max_heart_rate_bpm'].apply(lambda x: int(x))
staging_rides

/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1519149796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['max_heart_rate_bpm'] = staging_rides.groupby('ride_id', dropna=False)['heart_rate'].transform('max')
/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1519149796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['max_heart_rate_bpm'] = staging_rides['max_heart_rate_bpm'].apply(lambda x: int(x))


,ride_id,user_id,time,duration_secs,heart_rate,resistance,power,rpm,total_duration,total_power_kilojoules,max_heart_rate_bpm
0,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175
1,1,NaN,2022-10-06 15:49:21.453226,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175
2,1,4612.0,2022-10-06 15:49:21.953902,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175
3,1,NaN,2022-10-06 15:49:22.454586,1.0,NaN,0.0,NaN,NaN,0:08:37,13.51,175
4,1,NaN,2022-10-06 15:49:22.955260,NaN,0.0,NaN,3.044161,27.0,0:08:37,13.51,175
...,...,...,...,...,...,...,...,...,...,...,...
17632,17,NaN,2022-10-06 18:16:37.116680,NaN,87.0,NaN,8.207265,36.0,0:08:40,6.66,128
17633,17,NaN,2022-10-06 18:16:37.617353,519.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128
17634,17,NaN,2022-10-06 18:16:38.118052,NaN,87.0,NaN,7.837602,35.0,0:08:40,6.66,128
17635,17,NaN,2022-10-06 18:16:38.618768,520.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128


avg heart rate

In [16]:
staging_rides['avg_heart_rate_bpm'] = staging_rides.groupby('ride_id', dropna=False)['heart_rate'].transform('mean')
staging_rides['avg_heart_rate_bpm'] = staging_rides['avg_heart_rate_bpm'].apply(lambda x: int(x))
staging_rides

/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1606214273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['avg_heart_rate_bpm'] = staging_rides.groupby('ride_id', dropna=False)['heart_rate'].transform('mean')
/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1606214273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['avg_heart_rate_bpm'] = staging_rides['avg_heart_rate_bpm'].apply(lambda x: int(x))


,ride_id,user_id,time,duration_secs,heart_rate,resistance,power,rpm,total_duration,total_power_kilojoules,max_heart_rate_bpm,avg_heart_rate_bpm
0,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124
1,1,NaN,2022-10-06 15:49:21.453226,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124
2,1,4612.0,2022-10-06 15:49:21.953902,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124
3,1,NaN,2022-10-06 15:49:22.454586,1.0,NaN,0.0,NaN,NaN,0:08:37,13.51,175,124
4,1,NaN,2022-10-06 15:49:22.955260,NaN,0.0,NaN,3.044161,27.0,0:08:37,13.51,175,124
...,...,...,...,...,...,...,...,...,...,...,...,...
17632,17,NaN,2022-10-06 18:16:37.116680,NaN,87.0,NaN,8.207265,36.0,0:08:40,6.66,128,94
17633,17,NaN,2022-10-06 18:16:37.617353,519.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128,94
17634,17,NaN,2022-10-06 18:16:38.118052,NaN,87.0,NaN,7.837602,35.0,0:08:40,6.66,128,94
17635,17,NaN,2022-10-06 18:16:38.618768,520.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128,94


avg resistance

In [17]:
staging_rides['avg_resistance'] = staging_rides.groupby('ride_id', dropna=False)['resistance'].transform('mean')
staging_rides['avg_resistance'] = staging_rides['avg_resistance'].apply(lambda x: int(x))
staging_rides

/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1964614329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['avg_resistance'] = staging_rides.groupby('ride_id', dropna=False)['resistance'].transform('mean')
/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1964614329.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['avg_resistance'] = staging_rides['avg_resistance'].apply(lambda x: int(x))


,ride_id,user_id,time,duration_secs,heart_rate,resistance,power,rpm,total_duration,total_power_kilojoules,max_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance
0,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42
1,1,NaN,2022-10-06 15:49:21.453226,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42
2,1,4612.0,2022-10-06 15:49:21.953902,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42
3,1,NaN,2022-10-06 15:49:22.454586,1.0,NaN,0.0,NaN,NaN,0:08:37,13.51,175,124,42
4,1,NaN,2022-10-06 15:49:22.955260,NaN,0.0,NaN,3.044161,27.0,0:08:37,13.51,175,124,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17632,17,NaN,2022-10-06 18:16:37.116680,NaN,87.0,NaN,8.207265,36.0,0:08:40,6.66,128,94,39
17633,17,NaN,2022-10-06 18:16:37.617353,519.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128,94,39
17634,17,NaN,2022-10-06 18:16:38.118052,NaN,87.0,NaN,7.837602,35.0,0:08:40,6.66,128,94,39
17635,17,NaN,2022-10-06 18:16:38.618768,520.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128,94,39


avg rpm

In [18]:
staging_rides['avg_rpm'] = staging_rides.groupby('ride_id', dropna=False)['rpm'].transform('mean')
staging_rides['avg_rpm'] = staging_rides['avg_rpm'].apply(lambda x: int(x))
staging_rides

/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1084020979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['avg_rpm'] = staging_rides.groupby('ride_id', dropna=False)['rpm'].transform('mean')
/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1084020979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['avg_rpm'] = staging_rides['avg_rpm'].apply(lambda x: int(x))


,ride_id,user_id,time,duration_secs,heart_rate,resistance,power,rpm,total_duration,total_power_kilojoules,max_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm
0,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42,39
1,1,NaN,2022-10-06 15:49:21.453226,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42,39
2,1,4612.0,2022-10-06 15:49:21.953902,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42,39
3,1,NaN,2022-10-06 15:49:22.454586,1.0,NaN,0.0,NaN,NaN,0:08:37,13.51,175,124,42,39
4,1,NaN,2022-10-06 15:49:22.955260,NaN,0.0,NaN,3.044161,27.0,0:08:37,13.51,175,124,42,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17632,17,NaN,2022-10-06 18:16:37.116680,NaN,87.0,NaN,8.207265,36.0,0:08:40,6.66,128,94,39,41
17633,17,NaN,2022-10-06 18:16:37.617353,519.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128,94,39,41
17634,17,NaN,2022-10-06 18:16:38.118052,NaN,87.0,NaN,7.837602,35.0,0:08:40,6.66,128,94,39,41
17635,17,NaN,2022-10-06 18:16:38.618768,520.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128,94,39,41


start time

In [29]:
staging_rides['start_time'] = staging_rides.groupby('ride_id', dropna=False)['time'].transform('min')
staging_rides['start_time'] = staging_rides['start_time'].apply(lambda x: x.round(freq='S'))
staging_rides

/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1928088369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['start_time'] = staging_rides.groupby('ride_id', dropna=False)['time'].transform('min')
/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/1928088369.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['start_time'] = staging_rides['start_time'].apply(lambda x: x.round(freq='S'))


,ride_id,user_id,time,duration_secs,heart_rate,resistance,power,rpm,total_duration,total_power_kilojoules,max_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm,start_time,end_time
0,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42,39,2022-10-06 15:49:21,2022-10-06 15:58:00
1,1,NaN,2022-10-06 15:49:21.453226,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42,39,2022-10-06 15:49:21,2022-10-06 15:58:00
2,1,4612.0,2022-10-06 15:49:21.953902,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42,39,2022-10-06 15:49:21,2022-10-06 15:58:00
3,1,NaN,2022-10-06 15:49:22.454586,1.0,NaN,0.0,NaN,NaN,0:08:37,13.51,175,124,42,39,2022-10-06 15:49:21,2022-10-06 15:58:00
4,1,NaN,2022-10-06 15:49:22.955260,NaN,0.0,NaN,3.044161,27.0,0:08:37,13.51,175,124,42,39,2022-10-06 15:49:21,2022-10-06 15:58:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17632,17,NaN,2022-10-06 18:16:37.116680,NaN,87.0,NaN,8.207265,36.0,0:08:40,6.66,128,94,39,41,2022-10-06 18:07:58,2022-10-06 18:16:39
17633,17,NaN,2022-10-06 18:16:37.617353,519.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128,94,39,41,2022-10-06 18:07:58,2022-10-06 18:16:39
17634,17,NaN,2022-10-06 18:16:38.118052,NaN,87.0,NaN,7.837602,35.0,0:08:40,6.66,128,94,39,41,2022-10-06 18:07:58,2022-10-06 18:16:39
17635,17,NaN,2022-10-06 18:16:38.618768,520.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128,94,39,41,2022-10-06 18:07:58,2022-10-06 18:16:39


end time

In [28]:
staging_rides['end_time'] = staging_rides.groupby('ride_id', dropna=False)['time'].transform('max')
staging_rides['end_time'] = staging_rides['end_time'].apply(lambda x: x.round(freq='S'))
staging_rides

/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/2509912762.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['end_time'] = staging_rides.groupby('ride_id', dropna=False)['time'].transform('max')
/var/folders/df/2k5_vlhx3q9bg52x1jrdyzhh0000gn/T/ipykernel_77088/2509912762.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  staging_rides['end_time'] = staging_rides['end_time'].apply(lambda x: x.round(freq='S'))


,ride_id,user_id,time,duration_secs,heart_rate,resistance,power,rpm,total_duration,total_power_kilojoules,max_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm,start_time,end_time
0,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42,39,2022-10-06 15:49:21.453226,2022-10-06 15:58:00
1,1,NaN,2022-10-06 15:49:21.453226,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42,39,2022-10-06 15:49:21.453226,2022-10-06 15:58:00
2,1,4612.0,2022-10-06 15:49:21.953902,NaN,NaN,NaN,NaN,NaN,0:08:37,13.51,175,124,42,39,2022-10-06 15:49:21.453226,2022-10-06 15:58:00
3,1,NaN,2022-10-06 15:49:22.454586,1.0,NaN,0.0,NaN,NaN,0:08:37,13.51,175,124,42,39,2022-10-06 15:49:21.453226,2022-10-06 15:58:00
4,1,NaN,2022-10-06 15:49:22.955260,NaN,0.0,NaN,3.044161,27.0,0:08:37,13.51,175,124,42,39,2022-10-06 15:49:21.453226,2022-10-06 15:58:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17632,17,NaN,2022-10-06 18:16:37.116680,NaN,87.0,NaN,8.207265,36.0,0:08:40,6.66,128,94,39,41,2022-10-06 18:07:57.905291,2022-10-06 18:16:39
17633,17,NaN,2022-10-06 18:16:37.617353,519.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128,94,39,41,2022-10-06 18:07:57.905291,2022-10-06 18:16:39
17634,17,NaN,2022-10-06 18:16:38.118052,NaN,87.0,NaN,7.837602,35.0,0:08:40,6.66,128,94,39,41,2022-10-06 18:07:57.905291,2022-10-06 18:16:39
17635,17,NaN,2022-10-06 18:16:38.618768,520.0,NaN,30.0,NaN,NaN,0:08:40,6.66,128,94,39,41,2022-10-06 18:07:57.905291,2022-10-06 18:16:39


In [33]:
final_ride_columns = ['ride_id', 'user_id', 'start_time', 'end_time', 'total_duration', 'max_heart_rate_bpm', 'avg_heart_rate_bpm', 'avg_resistance', 'avg_rpm', 'total_power_kilojoules']

In [37]:
rides = staging_rides[final_ride_columns]
rides = rides.drop_duplicates()
rides = rides.dropna()
rides = rides.set_index('ride_id')
rides

,user_id,start_time,end_time,total_duration,max_heart_rate_bpm,avg_heart_rate_bpm,avg_resistance,avg_rpm,total_power_kilojoules
ride_id,,,,,,,,,
1,4612.0,2022-10-06 15:49:21,2022-10-06 15:58:00,0:08:37,175,124,42,39,13.51
2,4614.0,2022-10-06 15:58:01,2022-10-06 16:06:42,0:08:40,139,90,41,52,13.10
3,4614.0,2022-10-06 16:06:44,2022-10-06 16:15:25,0:08:40,180,89,41,40,14.53
4,4614.0,2022-10-06 16:15:27,2022-10-06 16:24:07,0:08:39,134,100,41,45,11.08
5,4614.0,2022-10-06 16:24:08,2022-10-06 16:32:50,0:08:40,166,114,41,43,8.76
6,4614.0,2022-10-06 16:32:51,2022-10-06 16:40:55,0:08:03,165,132,42,42,13.56
7,4614.0,2022-10-06 16:40:57,2022-10-06 16:49:34,0:08:36,171,125,41,43,10.42
8,4614.0,2022-10-06 16:49:35,2022-10-06 16:58:17,0:08:40,144,103,41,43,12.93
9,4616.0,2022-10-06 16:58:18,2022-10-06 17:06:58,0:08:39,142,41,40,45,12.42


## For future table insert

In [22]:
# cs.execute(f"""
#         CREATE OR REPLACE TABLE df_test_table
#         ("log" STRING,
#         "is_new_ride" BOOLEAN,
#         "is_info" BOOLEAN,
#         "is_system" BOOLEAN,
#         "date" DATETIME,
#         "heart_rate" INTEGER,
#         "resistance" INTEGER,
#         "rpm" INTEGER,
#         "power" FLOAT,
#         "user_id" STRING,
#         "name" STRING,
#         "gender" STRING,
#         "address" STRING,
#         "date_of_birth" FLOAT,
#         "email_address" STRING,
#         "height_cm" INTEGER,
#         "weight_kg" INTEGER,
#         "account_create_date" FLOAT,
#         "bike_serial" STRING,
#         "original_source" STRING
#         )
# """)

In [23]:
# write_pandas(conn, df, 'DF_TEST_TABLE')

In [24]:
# cs.execute(f'SELECT * FROM DF_TEST_TABLE WHERE "is_system" = TRUE').fetch_pandas_all()